**Objetivo do notebook**

Coletar os dados de quatro datasets com amostragem a cada 5 segundos e convertê-los em apenas um dataset de amostragem a cada 1 hora.

**Descrição dos datasets de entrada**

São 4 datasets de entrada. Cada um deles é resultado de uma simulação no software PSIM, utilizando as condições climáticas da cidade de Teresina-PI (base de dados Solcast) de todos os dias do ano de 2018 para se obter a temperatura da junção nas chaves de potência de um inversor conectado a um sistema fotovoltaico.

Estes datasets apresentam quatro variáveis cada:
* Time = momento de coleta da amostra na simulação, no formato ano-mês-dia-hora-minuto-segundo;
* Irradiance = valor de irradiância, em $kW/m^2$, na região de estudo (Teresina-PI);
* Ta = Temperatura ambiente da região de estudo;
* Tj = Temperatura da junção na chave (estimada pelo PSIM).

A diferença entre os datasets é a configuração de dispositivo utilizado como chave e de frequência de chaveamento:
* Dataset 0: IGBT (dispositivo) e 10kHz (frequência);
* Dataset 1: IGBT e 15kHz;
* Dataset 2: MOSFET e 10kHz;
* Dataset 3: MOSFET e 15kHz.

Essas informações de dispositivo e frequência também serão incluídas no dataset final como outras duas variáveis.

**Descrição do dataset produzido após processamento**

A variável Time (tempo) é importante para se realizar a média das variáveis de irradiância (irradiance), $T_a$ e $T_j$ em períodos de uma hora (reduzir a amostragem de 5 em 5 segundos para de hora em hora). Além disso, outra variável é computada, $\Delta T_j$: ela indica a variação entre os valores mínimo e máximo dentro do intervalo de uma hora.

Após isso, a variável de tempo é eliminada do dataframe.

Ao final, o dataframe é composto por seis variáveis no total.

**Destinação do dataset produzido**

O dataset produzido tem destinação a ser utilizado para treinamento de modelos (a partir de aprendizagem de máquina) que prevejam a Temperatura de Junção. Desta forma, das seis variáveis, quatro são destinadas a serem entradas para o modelo e duas, saída:
* Entrada: Irradiância, $T_a$, Frequência e Dispositivo;
* Saída: $T_j$ e $\Delta T_j$.

**Libraries**

In [41]:
# Libraries
import pandas as pd
import numpy as np
import pandas as pd
from sagemaker import get_execution_role 
from io import StringIO
import boto3
import botocore 

import warnings
warnings.simplefilter('ignore')

**Names of columns (Dataframe)**

Vetor de strings da head do dataframe (pandas) para renomear as variáveis do dataset para nomes mais representativos para o estudo:


In [5]:
columns = ['Time', 'Irradiance', 'Ta', 'Tj']

**Address, Devices and Frequency of Files**

Lista de dicionários, com cada dicionário contendo as informações para cada um dos datasets de entrada que serão lidos. Isso permite que estes datasets sejam lidos em um loop.

Cada dataset é resultado de uma simulação no software PSIM, utilizando as condições climáticas da cidade de Teresina-PI (base de dados Solcast) de todos os dias do ano de 2018 para se obter a temperatura da junção nas chaves de potência de um inversor conectado a um sistema fotovoltaico.

As informações contidas em cada dicionário:
* Endereço (address) do dataset a ser lido;
* Frequência de chaveamento (frequency) correspondente ao dataset lido;
* Dispositivo de chaveamento (device) correspondente ao dataset lido;
* Argumentos a serem utilizados na função pandas.read_csv().

In [33]:
files = [
    {
        'device' : 'IGBT',
        'freq'   : 10000,
        'address': 's3://resultados-ifsp-1000/2018/teresina/IGBT-CM100TU-12H-10kHz.txt',
        'kwargs' : {'header':None, 'sep': '  ', 'names': columns, 'chunksize': 100000}
    },
    {
        'device' : 'IGBT',
        'freq'   : 15000,
        'address': 's3://resultados-ifsp-1000/2018/teresina/IGBT_CM100_15kHz.txt',
        'kwargs' : {'header':None, 'sep': '  ', 'names': columns, 'chunksize': 200000}
    },
    {
        'device' : 'MOSFET',
        'freq'   : 10000,
        'address': 's3://resultados-ifsp-1000/2018/teresina/MOSFET- C3M0015065D-10kHz-tab-separated.txt',
        'kwargs' : {'header': 0, 'sep': '\t', 'names': columns, 'chunksize': 100000}
    },
    {
        'device' : 'MOSFET',
        'freq'   : 15000,
        'address': 's3://resultados-ifsp-1000/2018/teresina/MOSFET_C3M0015065D_15kHz.txt',
        'kwargs' : {'header':None, 'sep': '  ', 'names': columns, 'chunksize': 200000}
    }
]

**Function: Converting Dataframe with 100 or 200 thousand samples by day on one sample by hour**

Cada dataset possui 100 ou 200 mil amostras por dia da simulação: em torno de 4167 ou 8333 amostras por hora, respectivamente. Essa função reduz as amostras de um dia do banco de dados a 24 (uma amostra por hora):
* Irradiance, $T_a$ e $T_j$ = média no intervalo de uma hora;
* $\Delta T_j$ = $Tj_{máx} - Tj_{min}$.

In [34]:
# original to "by hour" function
def by_hour_function(df, freq, device):

  # converter index (0 a N_amostras) em horas (0 a 23h)
  div = np.round(len(df) / 24)
  np.floor(df.reset_index().index / div)
  df['Hour'] = np.floor(df.reset_index().index / div)

  by_hour_df = df.groupby('Hour').mean()[['Irradiance', 'Ta', 'Tj']].reset_index()
  by_hour_df['delta_Tj'] = df.groupby('Hour').max()['Tj'].reset_index()['Tj'] - df.groupby('Hour').min()['Tj'].reset_index()['Tj']

  by_hour_df.insert(3, 'f_sw', freq, False)
  by_hour_df.insert(4, 'Device', device, False)

  if len(by_hour_df.index) > 23:
    by_hour_df = by_hour_df[:24]

  return by_hour_df

**Function: Concatenate the 365 days Dataframes on one Dataframe**

* O carregamento de cada dataset é realizada por chunk = 100 ou 200 mil (a depender da quantidade de amostras diárias). Dessa forma, o dataset é lido dia a dia;
* Cada dia do dataset é reduzido a 24 amostras (uma por hora) pela função ```by_hour_function()```;
* As amostras de cada dia são novamente concatenadas em um dataset.

In [35]:
def concat_365(filepath, kwargs):

  # read file with chunks
  chunks =  pd.read_csv(filepath, **kwargs, engine='python')

  # declare a empty dataframe
  df_365 = pd.DataFrame()

  # for each chunk
  for i, chunk in enumerate(chunks):

    print(i, end='\r')
      
    # convert 100 or 200 thousand samples on 24 (hours) and concat on one dataframe
    df_365 = pd.concat([df_365,
                        by_hour_function(chunk, freq=files[num_file]['freq'], device=files[num_file]['device'])]
        )

  return df_365

**Loading datasets using chunk (large dataset) and converting to a reduced dataset**

* Cada dataset é reduzido pela função ```concat_365()```;
* Os quatro datasets são concatenados em um dataset final.

In [37]:
# concat all converted files

# get access to the execution role for the notebook instance
role = get_execution_role()

# declare a empty dataframe
final_df = pd.DataFrame()

for num_file in range(len(files)):
  filepath = files[num_file]['address']
  kwargs   = files[num_file]['kwargs']

  # concat all converted files (df_365)
  final_df = pd.concat([final_df,
                        concat_365(filepath, kwargs)]
    )

final_df

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


,Hour,Irradiance,Ta,f_sw,Device,Tj,delta_Tj
0,0.0,0.000000e+00,27.608333,10000,IGBT,29.081295,1.561258
1,1.0,0.000000e+00,27.179167,10000,IGBT,28.001805,0.870981
2,2.0,0.000000e+00,26.479123,10000,IGBT,27.179445,0.740504
3,3.0,0.000000e+00,26.091629,10000,IGBT,26.556674,0.493527
4,4.0,0.000000e+00,25.845771,10000,IGBT,26.160277,0.331715
...,...,...,...,...,...,...,...
19,19.0,3.501382e+02,28.741923,15000,MOSFET,65.811249,34.005698
20,20.0,5.696270e+01,28.483462,15000,MOSFET,54.522737,27.395597
21,21.0,1.252388e-01,28.366721,15000,MOSFET,38.075751,17.339361
22,22.0,8.684164e-08,28.316695,15000,MOSFET,30.786107,3.479579


**Remoção da coluna de tempo ('Hour')**

In [39]:
# remove the 'Hour' column
final_df.pop('Hour')

0      0.0
1      1.0
2      2.0
3      3.0
4      4.0
      ... 
19    19.0
20    20.0
21    21.0
22    22.0
23    23.0
Name: Hour, Length: 35040, dtype: float64

**Datraframe final produzido**

In [40]:
# show final dataframe
final_df

,Irradiance,Ta,f_sw,Device,Tj,delta_Tj
0,0.000000e+00,27.608333,10000,IGBT,29.081295,1.561258
1,0.000000e+00,27.179167,10000,IGBT,28.001805,0.870981
2,0.000000e+00,26.479123,10000,IGBT,27.179445,0.740504
3,0.000000e+00,26.091629,10000,IGBT,26.556674,0.493527
4,0.000000e+00,25.845771,10000,IGBT,26.160277,0.331715
...,...,...,...,...,...,...
19,3.501382e+02,28.741923,15000,MOSFET,65.811249,34.005698
20,5.696270e+01,28.483462,15000,MOSFET,54.522737,27.395597
21,1.252388e-01,28.366721,15000,MOSFET,38.075751,17.339361
22,8.684164e-08,28.316695,15000,MOSFET,30.786107,3.479579


**Dataset saving**

Salvamento do dataset em arquivo .csv.

In [51]:
bucket = 'resultados-ifsp-1000'  # already created on S3
csv_buffer = StringIO()
final_df.to_csv(csv_buffer, index=False)

s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'JTemp35k-teresina.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '5AFZXW3WZB1DHCQM',
  'HostId': 'eGNNYyc8B7Z992U8I97ne/WpXunnALtfxKB/x3DyfxrIt27VdvGuJCsaF7BPD8e1FAFsDEio/7U=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'eGNNYyc8B7Z992U8I97ne/WpXunnALtfxKB/x3DyfxrIt27VdvGuJCsaF7BPD8e1FAFsDEio/7U=',
   'x-amz-request-id': '5AFZXW3WZB1DHCQM',
   'date': 'Thu, 07 Dec 2023 00:50:43 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"c425915a2aa55f34c9698d0b7f1b35ae"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"c425915a2aa55f34c9698d0b7f1b35ae"',
 'ServerSideEncryption': 'AES256'}